

# Parsing IDMS schema syntax

## General References
* [Network Database, Relational DB, and Graph DB Compared](https://raima.com/network-database-relational-db-and-graph-db-compared/)
* IDMS is an implementation of the [The CODASYL Network Model](http://www.remote-dba.net/t_object_codasyl_network.htm) (contains info about Bachmann diagrams)

## IDMS-specific information
* [primer](https://www.sfu.ca/sasdoc/sashtml/idms/z0979730.htm)
* [IDMS schema and subschema syntax](https://manualzz.com/doc/o/rziew/ca-idms-database-administration-guide-schema-and-subschema-compilers)
* [CA IDMS Database Design Guide](https://ftpdocs.broadcom.com/cadocs/0/CA%20IDMS%2018%205%20User%20Bookshelf-ENU/Bookshelf_Files/PDF/IDMS_Database_Design_ENU.pdf)


## syntax components
* areas
* records - table
* elements - field
* sets

## Table of terms
 <img src="nonSQL_DB_equivalent_terms.png" alt="Table of non-SQL terms" style="float: left; width: 600px;"/>

# Import libraries

In [1]:
import pandas as pd
from pathlib import Path
import re
from collections import Counter

# List of all the IDMS subschema files we were provided

In [2]:
pwd

'/Users/ccoletta/projects/gsa_coe/usda/copybooks'

## Do a line count for each file we received

In [3]:
!wc -l  IDMS_schema_source_ALL/*

    2375 IDMS_schema_source_ALL/EMPSCHM-V100-SCHEMA-020623.txt
   16234 IDMS_schema_source_ALL/FARMS-V1-SCHEMA-020623.formatted.txt
   16234 IDMS_schema_source_ALL/FARMS-V1-SCHEMA-020623.txt
   16642 IDMS_schema_source_ALL/FARMS-V10-SCHEMA-020623.formatted.txt
   16642 IDMS_schema_source_ALL/FARMS-V10-SCHEMA-020623.txt
   25112 IDMS_schema_source_ALL/MCMMF01-SCHEMA-020623.txt
      47 IDMS_schema_source_ALL/RDUSE01-SCHEMA-020623.txt
    1599 IDMS_schema_source_ALL/RMS-SCHEMA-020623.formatted.txt
    1599 IDMS_schema_source_ALL/RMS-SCHEMA-020623.txt
    3387 IDMS_schema_source_ALL/SCHEMA-LISTING-020623.zip
    1779 IDMS_schema_source_ALL/SCMAC01-SCHEMA-020623.txt
      50 IDMS_schema_source_ALL/SCMACCT-SCHEMA-020623.txt
    1538 IDMS_schema_source_ALL/SCMFA01-SCHEMA-020623.txt
     263 IDMS_schema_source_ALL/SCMHN01-SCHEMA-020623.txt
   26281 IDMS_schema_source_ALL/SCMMF01-SCHEMA-020623.formatted.txt
   26281 IDMS_schema_source_ALL/SCMMF01-SCHEMA-020623.txt
   26039 IDMS_schema_source_A

# Examine the contents of one of the subschema files

In [4]:
!head -190 IDMS_schema_source_ALL/FARMS-V1-SCHEMA-020623.formatted.txt | tail -100

ADD
AREA NAME IS GUAR-AREA
    ESTIMATED PAGES ARE 0
    .
ADD
AREA NAME IS LNDR-AREA
    ESTIMATED PAGES ARE 0
    .
ADD
RECORD NAME IS CLIENT
    SHARE STRUCTURE OF RECORD CLIENT VERSION 15
        SYNONYM OF PRIMARY RECORD FARMS-CLIENT VERSION 15
    RECORD ID IS 100
    LOCATION MODE IS CALC USING ( CASE-NBR )
        DUPLICATES ARE NOT ALLOWED
    WITHIN AREA RCVB-AREA OFFSET 0 PERCENT FOR 100 PERCENT
    OWNER OF SET CLIENT-VARBL
        NEXT DBKEY POSITION IS 6
        PRIOR DBKEY POSITION IS 7
    OWNER OF SET CLIENT-ACCOUNTS
        NEXT DBKEY POSITION IS 4
        PRIOR DBKEY POSITION IS 5
    OWNER OF SET CLIENT-CLIENT
        NEXT DBKEY POSITION IS 3
    OWNER OF SET CLIENT-EQUITY
        NEXT DBKEY POSITION IS 1
    MEMBER OF SET IX-NME-OBLR
        INDEX DBKEY POSITION IS 8
    MEMBER OF SET IX-ID-NBR-OBLR
        INDEX DBKEY POSITION IS 9
    MEMBER OF SET IX-MLGOFC-CLIENT
        INDEX DBKEY POSITION IS 2
    MEMBER OF SET IX-MLGOFC-FSA
        INDEX DBKEY POSITION IS 1

# Create some regular expressions to help us parse schema text

In [6]:
#print( record_components[0] )

In [7]:
IS_pat = re.compile( r' IS ' )

# Define parsing functions

In [8]:
def DescriptorsSplitter( raw_descriptor_string : str ) -> pd.Series:
    """This function parses the descriptors after one single data element,
    including PICTURE, USAGE, ELEMENT LENGTH, POSITION, etc. Returns a single
    row's worth of element descriptors."""

    descriptors = [ _.strip() for _ in raw_descriptor_string.split( '\n' ) ]
    
    key_value_pairs = [ IS_pat.split( _ ) for _ in descriptors ]
    try:
        index, values = zip( *key_value_pairs )
    except:
        # empty series
        print( "\t\tproblem splitting these characteristics" )
        print( key_value_pairs )
        retval = pd.Series( dtype='object')
    else:
        retval = pd.Series( values, index=index )
    #print( retval )
    return retval

In [10]:
redefines_pattern = re.compile( r'REDEFINES' )
occurs_pattern = re.compile( r' OCCURS ' )

In [11]:
column_names = [ 'indent', 'data_level', 'element_name', 'raw_element_descriptors' ]
element_search_pat = re.compile( r'^(\s+)?(\d\d) (\S+)\n\s+(.*?)\n\s+\.', flags=re.MULTILINE | re.DOTALL )

In [12]:
RECORD_METADATA_pat = re.compile(
    r'RECORD NAME IS (\S+)\s+(.*)\.',
    flags=re.MULTILINE | re.DOTALL
)

In [13]:
RECORD_ID_pat = re.compile( r'RECORD ID IS (\S+)' )
WITHIN_AREA_pat = re.compile( r'WITHIN AREA (\S+) OFFSET (.*?) FOR (.*)' )
LOCATION_MODE_pat = re.compile( r'LOCATION MODE IS (.*?) USING \( (.*) \)' )
LOCATION_MODE_VIA_SET_pat = re.compile( r'LOCATION MODE IS VIA (.*?) SET' )

In [14]:
def FormatRecord( component_text : str, debug=False ) -> pd.DataFrame:
    """Takes the raw text of one record's worth of schema definition
    and parses all data elements from it."""
    
    if debug:
        print( "*" * 50 )
    
    # Process record metadata
    record_name, record_metadata_rawtext = \
        RECORD_METADATA_pat.match( component_text ).groups()
    record_id =                            \
        RECORD_ID_pat.search( record_metadata_rawtext ).group(1)
    area, offset, offset_extent =          \
        WITHIN_AREA_pat.search( record_metadata_rawtext ).groups()

    location_mode = None
    primary_key_name = None
    location_mode_set = ""

    m = LOCATION_MODE_pat.search( record_metadata_rawtext )
    if m:
        location_mode, primary_key_name =  m.groups()
    m = LOCATION_MODE_VIA_SET_pat.search( record_metadata_rawtext )
    if m:
        location_mode_set = m.group(1)

    # For some reason: the schema appends the record id to the primary key indicator
    if primary_key_name:
        remove_this = f'-{record_id}'
        primary_key_name = primary_key_name.replace( remove_this, "" )
    
    # Process record elements/fields/columns
    data_elements = element_search_pat.findall( component_text )

    data_elements = pd.DataFrame( data_elements, columns = column_names )
    data_elements['record'] = record_name
    data_elements['data_step'] = [ (1+int(_)) * 100 for _ in data_elements.index ]
    data_elements['indent'] = data_elements['indent'].apply( len )
    
    # Here, we add in an "IS" to REDEFINES and OCCURS descriptors to allow for
    # DescriptorSplitter() to work in a uniform way across all descriptors that
    # already use the "IS", e.g., "PICTURE", "ELEMENT LENGTH" etc.
    data_elements['raw_element_descriptors'] = \
        data_elements['raw_element_descriptors'].str.replace( redefines_pattern, 'REDEFINES IS' )

    data_elements['raw_element_descriptors'] = \
        data_elements['raw_element_descriptors'].str.replace( occurs_pattern, ' OCCURS IS ' )
    
    modifiers_df = data_elements['raw_element_descriptors'].apply( DescriptorsSplitter )
    
    data_elements = pd.concat( (data_elements, modifiers_df), axis=1 )
    data_elements = data_elements.set_index( 'record', append=True )
    
    data_elements['PRIMARY_KEY'] = ""
    if primary_key_name:
        data_elements.loc[ data_elements['element_name'] == primary_key_name, 'PRIMARY_KEY'] = 'PRIMARY KEY'
    
    data_elements['record_meta_IDMS_record_id'] = record_id
    data_elements['record_meta_WITHIN_AREA'] = area
    data_elements['record_meta_AREA_OFFSET'] = offset
    data_elements['record_meta_AREA_OFFSET_extent'] = offset_extent
    data_elements['record_meta_PK_in_SET'] = location_mode_set
    
    if debug:
        print( "record", record_name, "has", len( data_elements ), "elements." ,
             f'PRIMARY KEY="{primary_key_name}"' if primary_key_name else "" )
    
    return data_elements

In [15]:
def ScrapeRecordsAndElements( schema_source_path : Path, debug : bool = False ) -> pd.DataFrame:
    """Takes the path of one schema source file, parses out its component parts
    including SCHEMA, AREA, RECORD, and SET, and parses each record. Output is
    a PANDAS dataframe containing parsed info."""

    if debug:
        print( "=" * 50 )
    lines_df = pd.read_csv( schema_source_path, header=None )
    lines_df.columns = [ 'raw_line' ]
    lines_df[ 'stripped' ] = lines_df[ 'raw_line' ].str.slice( start = 5 )
    lines_df[ 'stripped' ] = lines_df[ 'stripped' ].str.rstrip()

    stripped_whitespace_file = Path( schema_source_path ).with_suffix( ".formatted.txt" )
    if debug:
        print( "writing", f'"{ str(stripped_whitespace_file) }"' )
    lines_df[ 'stripped' ].to_csv( stripped_whitespace_file, header=False, index=False )
    raw_text = stripped_whitespace_file.read_text()

    add_pat = re.compile( r'\nADD\n' )
    components = add_pat.split( raw_text )

    #len( components )
    # remove the initial ADD
    components[0] = components[0][4:]
    
    schema_name_ver_pat = re.compile( r'SCHEMA NAME IS (\S+) VERSION IS (\d+)' )
    schema_info = schema_name_ver_pat.search( components[0] ).groups()
    schema_name, schema_version = schema_info

    first_word_pat = re.compile( r'^(\S+)' )
    component_categories = [ first_word_pat.match( _ ).group(1) for _ in components ]
    c = Counter( component_categories )

    print( schema_info, "\n", c.most_common() )

    # Analyze Record components

    record_components = [ c for t, c in zip( component_categories, components ) if t == "RECORD" ]
    
    pivoted_record_data = pd.concat( [ FormatRecord( _, debug=debug ) for _ in record_components ] )
    pivoted_record_data = pivoted_record_data.swaplevel().sort_index()
    return pivoted_record_data

# INPUT THE SCHEMA FILE YOU WANT TO PARSE HERE:

In [16]:
retval = ScrapeRecordsAndElements( 'IDMS_schema_source_ALL/FARMS-V1-SCHEMA-020623.txt', debug=True )

writing "IDMS_schema_source_ALL/FARMS-V1-SCHEMA-020623.formatted.txt"
('FARMS', '1') 
 [('RECORD', 113), ('SET', 85), ('AREA', 20), ('SCHEMA', 1)]
**************************************************
record CLIENT has 76 elements. PRIMARY KEY="CASE-NBR"
**************************************************
record CLIENT-SFSI has 16 elements. 
**************************************************
record EQUITY has 34 elements. 
**************************************************
record INVSTR-INFO has 24 elements. PRIMARY KEY="ID-NBR-INVSTR"
**************************************************
record INVSTR-INFO-MISC has 3 elements. 
**************************************************
record PRTLSALE has 14 elements. 
**************************************************
record EQTYRVRS has 13 elements. 
**************************************************
record RENTL-CNTRL has 14 elements. PRIMARY KEY="GEOG-ST-CDE-RENTL-CNTRL"
**************************************************
record RENTL-TTL has 10 

In [17]:
retval.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2218 entries, ('ACCT-DATA', 0) to ('USERS', 12)
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   indent                          2218 non-null   int64 
 1   data_level                      2218 non-null   object
 2   element_name                    2218 non-null   object
 3   raw_element_descriptors         2218 non-null   object
 4   data_step                       2218 non-null   int64 
 5   USAGE                           2218 non-null   object
 6   ELEMENT LENGTH                  2154 non-null   object
 7   POSITION                        2218 non-null   object
 8   PICTURE                         1961 non-null   object
 9   REDEFINES                       23 non-null     object
 10  VALUE                           66 non-null     object
 11  PRIMARY_KEY                     2218 non-null   object
 12  record_meta_IDMS_record_

In [18]:
retval.head(30)

indent data_level          element_name  \
record                                                  
ACCT-DATA 0        0         05                LN-NBR   
          1        0         05                FD-CDE   
          2        4         10              FD-CDE-3   
          3        8         15              FD-CDE-2   
          4        8         15            FD-CDE-3RD   
          5        4         10            FD-CDE-4TH   
          6        0         05           KIND-CDE-LN   
          7        0         05         INT-RATE-NOTE   
          8        0         05     INT-RATE-NOTE-1ST   
          9        0         05          PYMT-TYP-CDE   
          10       0         05          DIR-PYMT-CDE   
          11       0         05      DTE-AMORTN-EFCTV   
          12       0         05        DSTR-DCLRD-CDE   
          13       4         10          DSTR-TYP-CDE   
          14       4         10         FY-DSTR-DCLRD   
          15       4         10        DSTR-DCLRD-NBR   
          16       0         05             MRG-CNTRL   
          17       0         05         DOCMT-TYP-CDE   
          18       0         05          DTE-OBLGN-LN   
          19       0         05        ASSTNC-TYP-CDE   
          20       0         05          LN-AMT-OBLGN   
          21       0         05  BEGNG-FRMR-RNCHR-CDE   
          22       0         05            COLLTL-CDE   
          23       0         05         CPN-PROCG-DTE   
          24       0         05     CASE-NBR-CHNG-CDE   
          25       0         05  PYMT-ASSTNC-METH-CDE   
          26       0         05         INT-RATE-PREV   
          27       0         05  INT-RATE-PREV-REDFND   
          28       0         05                FILLER   
ACQD-PROP 0        0         05      FD-CDE-ACQD-PROP   

                                        raw_element_descriptors  data_step  \
record                                                                       
ACCT-DATA 0   PICTURE IS  9(2)\n    USAGE IS DISPLAY\n    EL...        100   
          1   USAGE IS DISPLAY\n    ELEMENT LENGTH IS 4\n   ...        200   
          2   USAGE IS DISPLAY\n        ELEMENT LENGTH IS 3\...        300   
          3   PICTURE IS  9(2)\n            USAGE IS DISPLAY...        400   
          4   PICTURE IS  9(1)\n            USAGE IS DISPLAY...        500   
          5   PICTURE IS  9(1)\n        USAGE IS DISPLAY\n  ...        600   
          6   PICTURE IS  9(2)\n    USAGE IS DISPLAY\n    EL...        700   
          7   PICTURE IS  9(2)V9(4)\n    USAGE IS DISPLAY\n ...        800   
          8   REDEFINES IS INT-RATE-NOTE\n    PICTURE IS  V9...        900   
          9   PICTURE IS  9(1)\n    USAGE IS DISPLAY\n    EL...       1000   
          10  PICTURE IS  9(1)\n    USAGE IS DISPLAY\n    EL...       1100   
          11  PICTURE IS  9(06)\n    USAGE IS DISPLAY\n    E...       1200   
          12  USAGE IS DISPLAY\n    ELEMENT LENGTH IS 5\n   ...       1300   
          13  PICTURE IS  9(1)\n        USAGE IS DISPLAY\n  ...       1400   
          14  PICTURE IS  9(1)\n        USAGE IS DISPLAY\n  ...       1500   
          15  PICTURE IS  9(3)\n        USAGE IS DISPLAY\n  ...       1600   
          16  PICTURE IS  9(2)\n    USAGE IS DISPLAY\n    EL...       1700   
          17  PICTURE IS  X(1)\n    USAGE IS DISPLAY\n    EL...       1800   
          18  PICTURE IS  9(06)\n    USAGE IS DISPLAY\n    E...       1900   
          19  PICTURE IS  9(3)\n    USAGE IS DISPLAY\n    EL...       2000   
          20  PICTURE IS  S9(8)V99\n    USAGE IS COMP-3\n   ...       2100   
          21  PICTURE IS  X(01)\n    USAGE IS DISPLAY\n    E...       2200   
          22  PICTURE IS  9(1)\n    USAGE IS DISPLAY\n    EL...       2300   
          23  PICTURE IS  9(6)\n    USAGE IS DISPLAY\n    EL...       2400   
          24  PICTURE IS  9(01)\n    USAGE IS DISPLAY\n    E...       2500   
          25  PICTURE IS  9(1)\n    USAGE IS DISPLAY\n    EL...       2600   
     

# Here are all the records that belong to IDMS "user" sets

In [19]:
#retval.reset_index().groupby('record_meta_PK_in_SET')['record'].unique()

# Here is a list of all the DB areas and the list of records associated with them

In [20]:
retval.reset_index().groupby('record_meta_WITHIN_AREA')['record'].unique()

record_meta_WITHIN_AREA
ACQD-AREA                                     [ACQD-PROP, EASMNT]
APROPTN-AREA    [ALTMADJ, ALTMADJ-NEW, ALTMT, DALLOT, DALLOT-D...
CNTRL-AREA                               [RENTL-CNTRL, RENTL-TTL]
DISCRP-AREA     [ADPS-CNTRL, CASE, CRCLAIMS, DISCRP, DISCRP-MI...
GUAR-AREA                                                [GLBORR]
INVSTR-AREA           [INVSTR-DTL, INVSTR-INFO, INVSTR-INFO-MISC]
JURDCTN-AREA                                    [JURDCTN, ORDERS]
LEASE-AREA                                     [LESSEE, LSE-INFO]
LNDR-AREA                                                [GLLNDR]
LOOKUP-AREA     [APROPTN-LOOKUP, CDEJUNC, CDESTR, COHORT, CRRA...
OBLGN-AREA                  [CHECKS, INT-ASSTNC, INT-BDWN, OBLGN]
OUTPUT-AREA     [CK-CNTRL, CK-INFO, CK-INFO-FRADS, DAPROC, NOT...
PD-AREA                            [PD-ACCT-RVRSL, TRNSCTN-RVRSL]
PLEREP-AREA                                              [PLEREP]
RCVB-AREA       [ACCT-DATA, ADVANCE, AID, AMORTD-CST

In [21]:
# def RecordStats( grp ):
#     wanted_cols = ['record_meta_WITHIN_AREA', 'record', 'POSITION', 'record_meta_AREA_OFFSET', 'record_meta_AREA_OFFSET_extent' ]
#     return tuple( grp.loc[ grp.index[0], wanted_cols ].values )

# retval.reset_index().groupby( 'record' ).apply( RecordStats ).values

In [22]:
#retval.reset_index().groupby('record_meta_record_id')['record'].unique().apply( len ).unique()

# Cleanup

## Cleanup item 1: remove parentheses from valid values

In [23]:
retval['VALUE'].value_counts()

( 'S' )      10
( 'D' )      10
( 'C' )      10
( 'T' )       8
( 'U' )       8
( 'V' )       8
( '0' )       4
( '1' )       4
( SPACE )     2
( 'R' )       1
( 'A' )       1
Name: VALUE, dtype: int64

In [24]:
retval['VALUE'] = retval['VALUE'].str.extract( r' (\S+) ' )

In [25]:
retval['VALUE'].value_counts()

'S'      10
'D'      10
'C'      10
'T'       8
'U'       8
'V'       8
'0'       4
'1'       4
SPACE     2
'R'       1
'A'       1
Name: VALUE, dtype: int64

## Cleanup item 2: Reformat indents so copybooks look nice

In [26]:
retval['data_level'].value_counts()

05    1295
10     626
15     220
88      64
20      13
Name: data_level, dtype: int64

In [27]:
retval['indent'].value_counts()

0     1295
4      626
8      220
16      64
12      13
Name: indent, dtype: int64

In [28]:
retval['indent'] = (retval['data_level'].astype(int) // 5 )

In [29]:
retval.loc[ retval['indent'] >= 5, 'indent' ] = 5

In [30]:
retval['indent'] = retval['indent'] * 2

In [31]:
retval['indent'].value_counts()

2     1295
4      626
6      220
10      64
8       13
Name: indent, dtype: int64

## Cleanup item 3: remove leading spaces from PICTURE clause

In [32]:
retval['PICTURE'].values

array([' 9(2)', nan, nan, ..., ' 9(1)', ' XX', ' X'], dtype=object)

In [33]:
retval['PICTURE'] = retval['PICTURE'].str.strip()

# Add data formatting to the csv output to get it ready for ingestion by CreateCopyBooks notebook/script

Here we are just changing the tablenames of the output spreadsheet and adding dummy columns so the next script can run unedited.

In [34]:
table_index_dict = { table_name: i for i, table_name in enumerate( retval.index.levels[0] ) }

In [35]:
retval = retval.reset_index( 'record', drop=False )

In [36]:
retval['table_index'] = \
    [ table_index_dict[n] for n in retval['record'].values ]

In [37]:
retval['table_vers'] = 1

In [38]:
retval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2218 entries, 0 to 12
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   record                          2218 non-null   object
 1   indent                          2218 non-null   int64 
 2   data_level                      2218 non-null   object
 3   element_name                    2218 non-null   object
 4   raw_element_descriptors         2218 non-null   object
 5   data_step                       2218 non-null   int64 
 6   USAGE                           2218 non-null   object
 7   ELEMENT LENGTH                  2154 non-null   object
 8   POSITION                        2218 non-null   object
 9   PICTURE                         1961 non-null   object
 10  REDEFINES                       23 non-null     object
 11  VALUE                           66 non-null     object
 12  PRIMARY_KEY                     2218 non-null   ob

In [39]:
retval.head()

,record,indent,data_level,element_name,raw_element_descriptors,data_step,USAGE,ELEMENT LENGTH,POSITION,PICTURE,...,VALUE,PRIMARY_KEY,record_meta_IDMS_record_id,record_meta_WITHIN_AREA,record_meta_AREA_OFFSET,record_meta_AREA_OFFSET_extent,record_meta_PK_in_SET,OCCURS,table_index,table_vers
0,ACCT-DATA,2,05,LN-NBR,PICTURE IS 9(2)\n USAGE IS DISPLAY\n EL...,100,DISPLAY,2,1,9(2),...,NaN,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,NaN,0,1
1,ACCT-DATA,2,05,FD-CDE,USAGE IS DISPLAY\n ELEMENT LENGTH IS 4\n ...,200,DISPLAY,4,3,NaN,...,NaN,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,NaN,0,1
2,ACCT-DATA,4,10,FD-CDE-3,USAGE IS DISPLAY\n ELEMENT LENGTH IS 3\...,300,DISPLAY,3,3,NaN,...,NaN,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,NaN,0,1
3,ACCT-DATA,6,15,FD-CDE-2,PICTURE IS 9(2)\n USAGE IS DISPLAY...,400,DISPLAY,2,3,9(2),...,NaN,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,NaN,0,1
4,ACCT-DATA,6,15,FD-CDE-3RD,PICTURE IS 9(1)\n USAGE IS DISPLAY...,500,DISPLAY,1,5,9(1),...,NaN,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,NaN,0,1


In [40]:
reformatted_retval = retval.rename( 
    columns={ 
        'record': 'table_name',
        'element_name' : 'field_name',
        'USAGE' : 'end',
        'PICTURE' : 'data_type',
        'indent' : 'indent_space_count',
        'data_step' : 'declaration_step'
    } )

In [41]:
reformatted_retval['BLANK ON'] = ''
reformatted_retval['INDEXED BY'] = ''
reformatted_retval['OLQ'] = ''

In [42]:
reformatted_retval = reformatted_retval.drop( columns=['raw_element_descriptors'] )

In [43]:
reformatted_retval.to_csv( '2023-02-24_FSA_FARMS_schema_from_source.csv' )

In [44]:
#pd.set_option( 'display.max_rows', 100 )

In [45]:
# Two Python/PANDAS syntactical ways to select one whole table for inspection, if you want
#retval.loc[ 'ACCT-DATA' ]
#retval.loc[ ('ACCT-DATA', slice(None)), : ]

In [46]:
!head -30 2023-02-24_FSA_FARMS_schema_from_source.csv

,table_name,indent_space_count,data_level,field_name,declaration_step,end,ELEMENT LENGTH,POSITION,data_type,REDEFINES,VALUE,PRIMARY_KEY,record_meta_IDMS_record_id,record_meta_WITHIN_AREA,record_meta_AREA_OFFSET,record_meta_AREA_OFFSET_extent,record_meta_PK_in_SET,OCCURS,table_index,table_vers,BLANK ON,INDEXED BY,OLQ
0,ACCT-DATA,2,05,LN-NBR,100,DISPLAY,2,1,9(2),,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
1,ACCT-DATA,2,05,FD-CDE,200,DISPLAY,4,3,,,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
2,ACCT-DATA,4,10,FD-CDE-3,300,DISPLAY,3,3,,,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
3,ACCT-DATA,6,15,FD-CDE-2,400,DISPLAY,2,3,9(2),,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
4,ACCT-DATA,6,15,FD-CDE-3RD,500,DISPLAY,1,5,9(1),,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
5,ACCT-DATA,4,10,FD-CDE-4TH,600,DISPLAY,1,6,9(1),,,,215,RCVB-AREA,0 PERCENT,100 PERCENT,CLIENT-ACCOUNTS,,0,1,,,
6,ACCT-DATA,2,05,KIND-CDE-LN,7